# Creating a web scraper for Zillow.com
This scraper will search all of Zillow's listings for both recently sold and currently available single family homes in a specific state/county.

## Helpful videos:
https://www.youtube.com/watch?v=dRcvJRmqFHQ&ab_channel=WebScrapingwithAndy

https://www.youtube.com/watch?v=a3Cuq2csLWk

## Helpful websites:
curl.trillworks.com

In [512]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np
import regex as re
import time
import json

In [513]:
# Initial variables
page = 0
size = 0
county = 'brevard'
state = 'fl'
url = 'https://www.zillow.com/'
results = []

In [514]:
headers = {
    'authority': 'www.zillow.com',
    # 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # 'accept-encoding': 'gzip, deflate, br',
    'cache-control': 'max-age=0',
    'referer': 'https://www.zillow.com/',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    # 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }

# headers = {
#     'Accept': '*/*',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Accept-Language': 'en-US,en;q=0.9',
#     'upgrade-insecure-requests': '1',
#     'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
# }

# response = requests.get(url=url, params=params, headers=headers)
response = requests.get(url=url, headers=headers)
print(response.status_code)

200


In [515]:
while response.status_code == 200:
    page += 1

    url = 'https://www.zillow.com/%s-county-%s/' %(county, state)
    
    params = {
    'searchQueryState': '{"pagination":{"currentPage":%s},"isMapVisible":false,"mapBounds":{"west":-81.67492903125,"east":-79.69738996875,"south":27.568978152602924,"north":29.04158767448029},"mapZoom":9,"usersSearchTerm":"Brevard County, FL","regionSelection":[{"regionId":1556,"regionType":4}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}' %page
    }

    response = requests.get(url=url, params=params, headers=headers)
    time.sleep(0.25)
    if response.status_code != 200:
        break

    content = BeautifulSoup(response.text)

    deck = content.find('ul', {'class':'List-c11n-8-84-3__sc-1smrmqp-0 StyledSearchListWrapper-srp__sc-1ieen0c-0 doa-doM gKnRas photo-cards photo-cards_extra-attribution'})
    # If you inspect the "deck" variable, print(deck.prettify()), you will see a great many "timeouts". Not sure how to fix this...

    size = 0
    for card in deck.contents:
        size += 1
        script = card.find('script', {'type':'application/ld+json'}) # Excluding any advertisement cards. Script = None on ad cards.

        if script:
            script_json = json.loads(script.contents[0])
            
            try:
                square_feet = script_json['floorSize']['value']
            except:
                square_feet = ''

            try:
                latitude = script_json['geo']['latitude']
            except:
                latitude = ''
            
            try:
                longitude = script_json['geo']['longitude']
            except:
                longitude = ''
            
            try:
                address = script_json['name']
            except:
                address = ''
            
            try:
                homeDetailsURL = script_json['url']
            except:
                homeDetailsURL = ''

            try:
                price = card.find('span', {'class':'PropertyCardWrapper__StyledPriceLine-srp__sc-16e8gqd-1 iMKTKr'}).text
            except:
                price = ''

            try:
                beds = re.split('bds|ba| ', card.find('ul', {'class':'StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID'}).text)[0]
            except:
                beds = ''

            try:
                baths = re.split('bds|ba| ', card.find('ul', {'class':'StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID'}).text)[2]
            except:
                baths = ''

            results.append({
                'square_feet':square_feet,
                'latitude':latitude,
                'longitude':longitude,
                'address':address,
                'url':homeDetailsURL,
                'price':price,
                'beds':beds,
                'baths':baths
            })

    time.sleep(2)

df_housingInfo = pd.DataFrame.from_dict(results)

# Still need to figure out the timeout issue...
# Now you need to loop through all the home detail urls and add data to dataframe (age, zestimate, days on zillow, view, saves, has HOA )

Run request on first page
get all child elements from cards on first page
    from child elements is a "homedetails" url for all the specific data
loop through all homedetails urls on first page as second request set
    scrape necessary data from homedetails page into data frame
run request on second, third, ..., nth page